# Using Tools and Structured Outputs with Gemini

This notebook explores two powerful features for building capable AI agents with Large Language Models (LLMs): **Tools (Function Calling)** and **Structured Outputs**. We will use the `google-genai` library to interact with Google's Gemini models.

**Learning Objectives:**

1.  **Understand and implement tool use (function calling)** to allow an LLM to interact with external systems.
2.  **Enforce structured data formats (JSON)** from an LLM for reliable data extraction.
3.  **Leverage Pydantic models** to define and manage complex data structures for both function arguments and structured outputs, improving code robustness and clarity.

## 1. Setup

First, let's install the necessary Python libraries.

In [ ]:
!pip install -q google-generativeai pydantic python-dotenv

### Configure Gemini API Key

To use the Gemini API, you need an API key. 

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  Create a file named `.env` in the root of this project.
3.  Add the following line to the `.env` file, replacing `your_api_key_here` with your actual key:
    ```
    GEMINI_API_KEY="your_api_key_here"
    ```
The code below will load this key from the `.env` file.

In [11]:
import os
import json
from pathlib import Path

from dotenv import load_dotenv
from google import genai
from pydantic import BaseModel, Field
from typing import List

REPOSITORY_ROOT_DIR = Path().absolute().parent.parent
print(f"REPOSITORY_ROOT_DIR=`{REPOSITORY_ROOT_DIR}`")

try:
    load_dotenv(dotenv_path=REPOSITORY_ROOT_DIR / ".env")
except ImportError:
    print(
        "dotenv package not found. Please install it with 'pip install python-dotenv'"
    )

assert "GOOGLE_API_KEY" in os.environ, "`GOOGLE_API_KEY` is not set"

print("Environment variables loaded successfully.")

REPOSITORY_ROOT_DIR=`/Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents`
Environment variables loaded successfully.


### Initialize the Generative Model

We will use the `gemini-1.5-flash-latest` model, which is fast, cost-effective, and supports advanced features like tool use.

In [12]:
client = genai.Client()

## 2. Part 1: Using Tools (Function Calling)

LLMs are trained on text and can't perform actions in the real world on their own. **Tools** (or **Function Calling**) are the mechanism we use to bridge this gap. We provide the LLM with a list of available tools, and it can decide which one to use and with what arguments to fulfill a user's request.

The process is a loop:
1.  **You**: Send the LLM a prompt and a list of available tools.
2.  **LLM**: Responds with a `function_call` request, specifying the tool and arguments.
3.  **You**: Execute the requested function in your code.
4.  **You**: Send the function's output back to the LLM.
5.  **LLM**: Uses the tool's output to generate a final, user-facing response.

### Define Mock Tools

Let's create two simple, mocked functions. One simulates searching Google Drive, and the other simulates sending a Discord message. The function docstrings are crucial, as the LLM uses them to understand what each tool does.

In [13]:
def search_google_drive(query: str) -> str:
    """
    Searches for a file on Google Drive and returns its content or a summary.

    Args:
        query (str): The search query to find the file, e.g., 'Q3 earnings report'.

    Returns:
        str: A JSON string representing the search results, including file names and summaries.
    """

    print(f"---> Searching Google Drive for: '{query}'")
    # In a real scenario, this would interact with the Google Drive API.
    # Here, we mock the response for demonstration.
    if "q3 earnings report" in query.lower():
        return json.dumps(
            {
                "files": [
                    {
                        "name": "Q3_Earnings_Report_2024.pdf",
                        "id": "file12345",
                        "summary": "The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations.",
                    }
                ]
            }
        )
    else:
        return json.dumps({"files": []})


def send_discord_message(channel_id: str, message: str) -> str:
    """
    Sends a message to a specific Discord channel.

    Args:
        channel_id (str): The ID of the channel to send the message to, e.g., '#finance'.
        message (str): The content of the message to send.

    Returns:
        str: A JSON string confirming the action, e.g., '{"status": "success"}'.
    """

    print(f"---> Sending message to Discord channel '{channel_id}': '{message}'")
    # Mocking a successful API call
    return json.dumps(
        {
            "status": "success",
            "channel": channel_id,
            "message_preview": f"{message[:50]}...",
        }
    )

### Running the Tool Use Loop

Now, let's create a scenario where we ask the agent to perform a multi-step task: find a report and then communicate its findings.

In [27]:
from google.genai import types

# The user's request that requires tool use
prompt = "Please find the Q3 earnings report on Google Drive and send a summary of it to the #finance channel on Discord."

# Define the function declarations explicitly
search_google_drive_declaration = {
    "name": "search_google_drive",
    "description": "Searches for a file on Google Drive and returns its content or a summary.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to find the file, e.g., 'Q3 earnings report'.",
            }
        },
        "required": ["query"],
    },
}

send_discord_message_declaration = {
    "name": "send_discord_message",
    "description": "Sends a message to a specific Discord channel.",
    "parameters": {
        "type": "object",
        "properties": {
            "channel_id": {
                "type": "string",
                "description": "The ID of the channel to send the message to, e.g., '#finance'.",
            },
            "message": {
                "type": "string",
                "description": "The content of the message to send.",
            },
        },
        "required": ["channel_id", "message"],
    },
}

# Create a lookup for the actual Python functions
tool_functions = {
    func.__name__: func for func in [search_google_drive, send_discord_message]
}

tools = [
    types.Tool(
        function_declarations=[
            types.FunctionDeclaration(**search_google_drive_declaration),
            types.FunctionDeclaration(**send_discord_message_declaration),
        ]
    )
]
config = types.GenerateContentConfig(
    tools=tools,
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
)

# 1. First call to the model
print(f"User Prompt: {prompt}")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config=config,
)
response_message = response.candidates[0].content.parts[0]

print(f"\nModel's first response: {response_message.function_call}")

# Keep a list of messages to send back to the model
messages = [response.candidates[0].content]

# Loop to handle multiple function calls
max_iterations = 3
while hasattr(response_message, "function_call") and max_iterations > 0:
    function_call = response_message.function_call
    function_name = function_call.name

    # 2. Execute the function requested by the model
    if function_name in tool_functions:
        selected_function = tool_functions[function_name]
        args = {key: value for key, value in function_call.args.items()}
        tool_result = selected_function(**args)
    else:
        raise ValueError(f"Unknown function call: {function_name}")

    # 3. Send the result back to the model
    print(f"\nSending tool result back to model: {tool_result}")
    function_response_part = types.Part(
        function_response=types.FunctionResponse(
            name=function_name, response=json.loads(tool_result)
        )
    )
    messages.append(function_response_part)

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=messages,
        config=config,
    )

    # The model may call another function or return a text response
    response_message = response.candidates[0].content.parts[0]
    messages.append(response.candidates[0].content)

    print(f"\nModel's next response: {response_message}")

    max_iterations -= 1

# 4. Print the final, user-facing answer
print("\n--- Final Agent Response ---")
print(response.text)


User Prompt: Please find the Q3 earnings report on Google Drive and send a summary of it to the #finance channel on Discord.

Model's first response: id=None args={'query': 'Q3 earnings report'} name='search_google_drive'
---> Searching Google Drive for: 'Q3 earnings report'

Sending tool result back to model: {"files": [{"name": "Q3_Earnings_Report_2024.pdf", "id": "file12345", "summary": "The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations."}]}

Model's next response: video_metadata=None thought=None inline_data=None file_data=None thought_signature=None code_execution_result=None executable_code=None function_call=FunctionCall(id=None, args={'channel_id': '#finance', 'message': 'Q3 Earnings Report Summary: Revenue increased by 20%, user engagement grew by 15%, beating expectations. File ID: file12345'}, name='send_discord_message') function_response=None text=None
---> Sending message to Discord channel '#finance': 'Q3 Ear


Model's next response: video_metadata=None thought=None inline_data=None file_data=None thought_signature=None code_execution_result=None executable_code=None function_call=FunctionCall(id=None, args={'message': 'Q3 Earnings Report Summary: Revenue increased by 20%, user engagement grew by 15%, beating expectations. Google Drive File ID: file12345', 'channel_id': '#finance'}, name='send_discord_message') function_response=None text=None

--- Final Agent Response ---
None


## 3. Part 2: Structured Outputs with JSON

Sometimes, you don't need the LLM to take an action, but you need its output in a specific, machine-readable format. Forcing the output to be JSON is a common way to achieve this.

We can instruct the model to do this by:
1.  **Prompting**: Clearly describe the desired JSON structure in the prompt.
2.  **Configuration**: Setting `response_mime_type` to `"application/json"` in the generation configuration, which forces the model's output to be a valid JSON object.

### Example: Extracting Metadata from a Document

Let's imagine we have a markdown document and we want to extract key information like a summary, tags, and keywords into a clean JSON object.

In [30]:
document = """
# Article: The Rise of AI Agents

This article discusses the recent advancements in AI, focusing on autonomous agents. 
We explore how Large Language Models (LLMs) are moving beyond simple text generation 
to perform complex, multi-step tasks. Key topics include the ReAct framework, 
the importance of tool use, and the challenges of long-term planning. The future 
of software development may be significantly impacted by these new AI paradigms.
"""

prompt = f"""
Please analyze the following document and extract metadata from it. 
The output must be a single, valid JSON object with the following structure:
{{ "summary": "A concise summary of the article.", "tags": ["list", "of", "relevant", "tags"], "keywords": ["list", "of", "key", "concepts"] }}

Document:
--- 
{document}
--- 
"""

# Configure the model to output JSON
config = types.GenerateContentConfig(response_mime_type="application/json")

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt, config=config
)

print("--- Raw LLM Output ---")
print(response.text)

# You can now reliably parse the JSON string
metadata_obj = json.loads(response.text)

print("\n--- Parsed JSON Object ---")
print(metadata_obj)

--- Raw LLM Output ---
{
  "summary": "This article discusses the rise of AI agents and their ability to perform complex tasks using Large Language Models (LLMs). It covers the ReAct framework, tool use, and long-term planning challenges, suggesting a significant impact on the future of software development.",
  "tags": ["AI", "agents", "LLMs", "autonomous agents", "software development"],
  "keywords": ["ReAct framework", "tool use", "long-term planning", "artificial intelligence", "large language models"]
}

--- Parsed JSON Object ---
{'summary': 'This article discusses the rise of AI agents and their ability to perform complex tasks using Large Language Models (LLMs). It covers the ReAct framework, tool use, and long-term planning challenges, suggesting a significant impact on the future of software development.', 'tags': ['AI', 'agents', 'LLMs', 'autonomous agents', 'software development'], 'keywords': ['ReAct framework', 'tool use', 'long-term planning', 'artificial intelligence',

## 4. Part 3: Structured Outputs with Pydantic

While prompting for JSON is effective, it can be fragile. A more robust and modern approach is to use **Pydantic**. Pydantic allows you to define data structures as Python classes. This gives you:

- **A single source of truth**: The Pydantic model defines the structure.
- **Automatic schema generation**: You can easily generate a JSON Schema from the model.
- **Data validation**: You can validate the LLM's output against the model to ensure it conforms to the expected structure and types.

Let's recreate the previous example using Pydantic.

In [31]:
class DocumentMetadata(BaseModel):
    """A class to hold structured metadata for a document."""

    summary: str = Field(description="A concise, 1-2 sentence summary of the document.")
    tags: List[str] = Field(
        description="A list of 3-5 high-level tags relevant to the document."
    )
    keywords: List[str] = Field(
        description="A list of specific keywords or concepts mentioned."
    )


### Method 1: Injecting Pydantic Schema into the Prompt

We can generate a JSON Schema from our Pydantic model and inject it directly into the prompt. This is a more formal way of telling the LLM what structure to follow.

In [32]:
schema = DocumentMetadata.model_json_schema()

prompt = f"""
Please analyze the following document and extract metadata from it. 
The output must be a single, valid JSON object that conforms to the following JSON Schema:
```json
{json.dumps(schema, indent=2)}
```

Document:
--- 
{document}
--- 
"""

config = types.GenerateContentConfig(response_mime_type="application/json")
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt, config=config
)

print("--- Raw LLM Output ---")
print(response.text)

# Now, we can validate the output with Pydantic
try:
    validated_metadata = DocumentMetadata.model_validate_json(response.text)
    print("\n--- Pydantic Validated Object ---")
    print(validated_metadata)
    print("\nValidation successful!")
except Exception as e:
    print(f"\nValidation failed: {e}")

--- Raw LLM Output ---
{
  "summary": "The article discusses the rise of AI agents, focusing on autonomous agents and the use of Large Language Models (LLMs) for complex tasks. Key aspects include the ReAct framework, tool use, and long-term planning challenges.",
  "tags": [
    "AI Agents",
    "Large Language Models",
    "Autonomous Systems",
    "Artificial Intelligence"
  ],
  "keywords": [
    "LLMs",
    "ReAct framework",
    "tool use",
    "long-term planning",
    "autonomous agents"
  ]
}

--- Pydantic Validated Object ---
summary='The article discusses the rise of AI agents, focusing on autonomous agents and the use of Large Language Models (LLMs) for complex tasks. Key aspects include the ReAct framework, tool use, and long-term planning challenges.' tags=['AI Agents', 'Large Language Models', 'Autonomous Systems', 'Artificial Intelligence'] keywords=['LLMs', 'ReAct framework', 'tool use', 'long-term planning', 'autonomous agents']

Validation successful!


### Method 2: Using a Pydantic Model as a Tool

A more elegant and powerful pattern is to treat our Pydantic model *as a tool*. We can ask the model to "call" this Pydantic tool, and the arguments it generates will be our structured data.

This combines the power of function calling with the robustness of Pydantic for structured data extraction. It's the recommended approach for complex data extraction tasks.

In [35]:
# The Pydantic class 'DocumentMetadata' is now our 'tool'
extraction_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="extract_metadata",
            description="Extracts structured metadata from a document.",
            parameters=DocumentMetadata.model_json_schema(),
        )
    ]
)
config = types.GenerateContentConfig(
    tools=[extraction_tool],
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
)

prompt = f"""
Please analyze the following document and extract its metadata.

Document:
--- 
{document}
--- 
"""

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt, config=config
)
response_message = response.candidates[0].content.parts[0]

if hasattr(response_message, "function_call"):
    function_call = response_message.function_call
    print("--- Function Call from LLM ---")
    print(function_call)

    # The arguments are our structured data
    metadata_args = {key: val for key, val in function_call.args.items()}

    # We can now validate and use this data with our Pydantic model
    try:
        validated_metadata = DocumentMetadata(**metadata_args)
        print("\n--- Pydantic Validated Object ---")
        print(validated_metadata)
        print(f"\nSummary: {validated_metadata.summary}")
        print(f"Tags: {validated_metadata.tags}")
    except Exception as e:
        print(f"\nValidation failed: {e}")
else:
    print("The model did not call the extraction tool.")

--- Function Call from LLM ---
id=None args={'summary': 'The article discusses advancements in AI, focusing on autonomous agents and how LLMs are moving beyond text generation to perform complex tasks.', 'tags': ['AI', 'Autonomous Agents', 'LLMs'], 'keywords': ['AI Agents', 'Large Language Models', 'ReAct framework', 'tool use', 'long-term planning']} name='extract_metadata'

--- Pydantic Validated Object ---
summary='The article discusses advancements in AI, focusing on autonomous agents and how LLMs are moving beyond text generation to perform complex tasks.' tags=['AI', 'Autonomous Agents', 'LLMs'] keywords=['AI Agents', 'Large Language Models', 'ReAct framework', 'tool use', 'long-term planning']

Summary: The article discusses advancements in AI, focusing on autonomous agents and how LLMs are moving beyond text generation to perform complex tasks.
Tags: ['AI', 'Autonomous Agents', 'LLMs']


### Method 3: Using a Pydantic Model as direct Output

In [38]:
config = types.GenerateContentConfig(
    response_mime_type="application/json",
    response_schema=DocumentMetadata
)

prompt = f"""
Please analyze the following document and extract its metadata.

Document:
--- 
{document}
--- 
"""

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt, config=config
)
response.parsed

DocumentMetadata(summary='This article examines the progress of AI agents, particularly their ability to handle complex tasks using Large Language Models. It highlights the ReAct framework, the significance of utilizing tools, and the difficulties associated with long-term planning in AI.', tags=['AI Agents', 'Large Language Models', 'Autonomous Systems', 'Software Development'], keywords=['AI', 'LLMs', 'ReAct framework', 'tool use', 'long-term planning'])